# Simulating voting rules (Python 3.10)

## Introduction

Voting rules can be confusing. To illustrate the possible voting rules as discussed in the lectures, we simulate the voting rules and the effects they can have. In this demo, "agent" and "voter" is used interchangably.

Based on information provided in lecture slides by Federico Fioravanti.

In [159]:
import random
import numpy as np
import string
from collections import Counter

## Model definition


### Defining functions

We first define the functions `create_voters` and `create_alternatives` to create specific sets $N$ and $A$, respectively.

In [160]:
def create_voters(n_voters: int = 0) -> list[int]:
    """
    Create a list of voters of size `n_voters`.
    If `n_voters` is equal to 0, a random number of voters in the range 
    [3, 10] will be created.
    """

    assert isinstance(n_voters, int), 'Number of voters must be an integer'
    assert n_voters >= 0, 'Number of voters must be zero or positive'

    if n_voters == 0:
        n_voters = random.randint(3, 10)

    return [i for i in range(n_voters)]


def create_alternatives(n_alternatives: int = 0) -> list[str]:
    """
    Create a list of alternatives of size `n_alternatives`.
    If `n_alternatives` is equal to 0, a random number of alternatives in the range 
    [3, 10] will be created.
    """

    assert isinstance(n_alternatives, int), 'Number of alternatives must be an integer'
    assert n_alternatives >= 0, 'Number of alternatives must be zero or positive'
    assert n_alternatives <= 26, 'Number of alternatives must be less than or equal to 26'

    if n_alternatives == 0:
        n_alternatives = random.randint(3, 10)

    return list(string.ascii_lowercase[:n_alternatives])


print(f'{create_voters(5)=}')
print(f'{create_alternatives(3)=}')

create_voters(5)=[0, 1, 2, 3, 4]
create_alternatives(3)=['a', 'b', 'c']


We define a function `create_preference_profile` for creating preference profiles. For each agent $i \in N$ it creates a preference $\succ_i$ containing alternatives from the set $A$:

In [161]:
def create_profile(voters: list[int], alternatives: list[str]) -> dict:
    """
    Create a preference profile for a given number of voters and alternatives.
    Randomized on each call.
    """
    
    profile = dict()

    for voter in voters:
        profile[voter] = ''.join(random.sample(alternatives, k=len(alternatives)))
        
    return profile

Functions for getting the top choice of a voter $i$ and the bottom choice: $top(\succ_i) = a_{i_1}$ and $bot(\succ_i) = a_{i_{|A|}}$

In [162]:
def top_choice(profile: dict, voter: int) -> str:
    """
    Returns the top choice of a voter.
    """
    
    return profile[voter][0]

def bottom_choice(profile: dict, voter: int) -> str:
    """
    Returns the bottom choice of a voter.
    """
    
    return profile[voter][-1]

## Description of a sample preference profile

In [163]:
# sample preference profile for 4 voters and 3 alternatives
n_voters = 4
n_alternatives = 3
print(f'{n_voters=}')
print(f'{n_alternatives=}')

voters = create_voters(n_voters)
alternatives = create_alternatives(n_alternatives)
profile = create_profile(voters, alternatives)
print('\nRandomized sample profile:')
print(f'{profile=}')


print('\nPreferences by voter:')

for voter in voters:
    print(f'Voter {voter}: {profile[voter]}')


# count the number of each preference
preference_count = Counter(profile.values())

print('\nPreferences by frequency:')

for preference, count in preference_count.most_common():
    print(f'{preference}: {count}')

n_voters=4
n_alternatives=3

Randomized sample profile:
profile={0: 'cba', 1: 'acb', 2: 'cab', 3: 'abc'}

Preferences by voter:
Voter 0: cba
Voter 1: acb
Voter 2: cab
Voter 3: abc

Preferences by frequency:
cba: 1
acb: 1
cab: 1
abc: 1


## The dictatorship rule $F_{DCT}$

For any profile $R$: $F_{DCT}(R) = top(\succ_i)$, with voter $i \in N$ being the "dictator".

In [164]:
def dictatorship_rule(profile: dict, dictator: int) -> str:
    """
    This rule returns the top choice of the dictator specified.
    """
    
    return top_choice(profile, dictator)

In [165]:
print(f'{n_voters=}')
print(f'{n_alternatives=}')
print(f'{profile=}')

dictator = random.choice(voters)
print(f'\nDictator: {dictator}')

winner = dictatorship_rule(profile, dictator)
print(f'Dictatorship rule winner: {winner}')

n_voters=4
n_alternatives=3
profile={0: 'cba', 1: 'acb', 2: 'cab', 3: 'abc'}

Dictator: 3
Dictatorship rule winner: a


## The majority rule $F_{MAJ}$

The majority rule states that for any profile $R$: $F_{MAJ}(R) = x$, such that $top(\succ_i) = x$ for a strict majority of voters in $N$. A strict majority is the the half of the voters plus one. 

The majority rule does not always have an outcome. In that case, the function `majority_rule` returns `None`. This can happen when `n_voters` is even and/or `n_alternatives` is greater than 2.

In [166]:
def majority_rule(profile: dict) -> str | None:
    """
    This rule returns the majority top choice.
    If there is no majority, it returns None.
    """
    
    majority = len(profile) // 2 + 1

    top_choices = [top_choice(profile, voter) for voter in profile]
    top_choice_count = Counter(top_choices)

    for top, count in top_choice_count.items():
        if count >= majority:
            return top
        
    return None

In [167]:
print(f'{n_voters=}')
print(f'{n_alternatives=}')
print(f'{profile=}')

winner = majority_rule(profile)
print(f'\nMajority rule winner: {winner}')

n_voters=4
n_alternatives=3
profile={0: 'cba', 1: 'acb', 2: 'cab', 3: 'abc'}

Majority rule winner: None


## The plurality rule $F_{PLU}$

The plurality rule states that for any profile $R$: $F_{PLU}(R) = \text{argmax}_{x \in A}p_R(x)$, where $p_R(x) = | \{ i \in N | top(\succ_i) = x \} |$. That is, the plurality rule extracts the alternatives that are the most at the top of the voters' preferences.

This implementation of `plurality_rule` returns a random winner in case of a tie. This results in the fact that this rule always has a winner.

In [168]:
def plurality_rule(profile: dict) -> str:
    """
    This rule returns the plurality top choice.
    If there are ties, return a random winner.
    """
    
    # construct a list of all the top choices, and count them
    top_choices = [top_choice(profile, voter) for voter in profile]
    choice_votes = Counter(top_choices)

    # find the maximum number of votes
    max_votes = max(choice_votes.values())

    # get winners
    winners = [c for c, v in choice_votes.items() if v == max_votes]
    
    return random.choice(winners)

In [169]:
print(f'{n_voters=}')
print(f'{n_alternatives=}')
print(f'{profile=}')

print('\nPlurality rule:')

winner = plurality_rule(profile)
print(f'Winner: {winner}')

n_voters=4
n_alternatives=3
profile={0: 'cba', 1: 'acb', 2: 'cab', 3: 'abc'}

Plurality rule:
Winner: a


### Duverger's law

Duverger's law states that:

> A single-ballot plurality-rule election structured within single-member districts tends to favor a two-party system

The two-party system in the United States is a result of Duverger's law. The president is chosen with this voting rule. To illustrate how unfavorable this voting system is, we will simulate a country with 100 citizens and varying between 5, 4, 3 and 2 alternatives. For simplicity, we discard the districts and imagine the entire country consists of a single district.

#### Plurality with 5 alternatives

In [170]:
random.seed(27)

n_voters = 100
n_alternatives = 5
voters = create_voters(n_voters)
alternatives = create_alternatives(n_alternatives)
profile = create_profile(voters, alternatives)

print('\nPreferences by frequency:')

preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

winner = plurality_rule(profile)
winner_on_top = sum([1 for voter in profile if top_choice(profile, voter) == winner])
winner_on_bottom = sum([1 for voter in profile if bottom_choice(profile, voter) == winner])
winner_on_bottom_2 = sum([1 for voter in profile if winner in profile[voter][-2:]])

print(f'\nPlurality rule winner: {winner}')
print(f'Top vote: {winner_on_top} times')
print(f'Bottom vote: {winner_on_bottom} times')
print(f'In bottom 2 votes: {winner_on_bottom_2} times')



Preferences by frequency:
cadeb:  6   bcdea:  3   cbdae:  3   debac:  3   eadbc:  3   edcab:  3   
acdbe:  2   daceb:  2   dacbe:  2   adbec:  2   dceab:  2   bdcae:  2   
eadcb:  2   cedab:  2   daebc:  2   ebadc:  2   cdabe:  2   bdcea:  2   
bcead:  2   acedb:  2   bdaec:  2   edbac:  2   bcaed:  2   bdeca:  2   
cdeba:  2   baced:  2   bdace:  2   edcba:  2   eacbd:  2   decab:  2   
dcbae:  1   cdaeb:  1   ebcad:  1   cdbea:  1   caedb:  1   cabed:  1   
eabdc:  1   badec:  1   bcdae:  1   edbca:  1   dbace:  1   becda:  1   
dcaeb:  1   eacdb:  1   abdce:  1   cedba:  1   dbcae:  1   aedbc:  1   
aecdb:  1   ecbda:  1   acdeb:  1   bceda:  1   debca:  1   deacb:  1   
cbade:  1   edabc:  1   edacb:  1   adcbe:  1   aebcd:  1   bedac:  1   
daecb:  1   

Plurality rule winner: b
Top vote: 24 times
Bottom vote: 30 times
In bottom 2 votes: 51 times


As we can see, the plurality rule winner for this profile is alternative $b$. However, even though $b$ reached the top of the voter's preference 24 times, they were at the bottom of the preference of 30 voters. This shows that the plurality rule can provide terrible winners that do not align with many people's preferences. Even worse, however, is that in 51 out of 100 voters, the winner $b$ was either one of the bottom two alternatives in their preference. The votes for alternatives that were less despised by the population were split, and as a result, the winner with the most votes was $b$.

#### Plurality with 4 alternatives

In [171]:
random.seed(706)

n_voters = 100
n_alternatives = 4
voters = create_voters(n_voters)
alternatives = create_alternatives(n_alternatives)
profile = create_profile(voters, alternatives)

print('\nPreferences by frequency:')

preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

winner = plurality_rule(profile)
winner_on_top = sum([1 for voter in profile if top_choice(profile, voter) == winner])
winner_on_bottom = sum([1 for voter in profile if bottom_choice(profile, voter) == winner])
winner_on_bottom_2 = sum([1 for voter in profile if winner in profile[voter][-2:]])

print(f'\nPlurality rule winner: {winner}')
print(f'Top vote: {winner_on_top} times')
print(f'Bottom vote: {winner_on_bottom} times')
print(f'In bottom 2 votes: {winner_on_bottom_2} times')



Preferences by frequency:
dabc:  8   cdba:  7   cadb:  7   abcd:  6   badc:  6   dacb:  5   
cabd:  5   dbca:  5   cdab:  5   dcba:  5   adbc:  4   bcda:  4   
bcad:  4   bacd:  4   abdc:  4   bdca:  4   cbad:  4   acbd:  3   
dcab:  3   dbac:  3   cbda:  2   adcb:  1   bdac:  1   

Plurality rule winner: c
Top vote: 30 times
Bottom vote: 26 times
In bottom 2 votes: 51 times


In the above voting simulation, we can again see that the majority of voters have written down the winner ($c$) to be in the bottom half of their preference. This time however, the amount of times that alternative $c$ is at the top of preferences is more than the amount of times $c$ is at the bottom of preferences.

#### Plurality with 3 alternatives

In [172]:
random.seed(91)

n_voters = 100
n_alternatives = 3
voters = create_voters(n_voters)
alternatives = create_alternatives(n_alternatives)
profile = create_profile(voters, alternatives)

print('\nPreferences by frequency:')

preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

winner = plurality_rule(profile)
winner_on_top = sum([1 for voter in profile if top_choice(profile, voter) == winner])
winner_on_bottom = sum([1 for voter in profile if bottom_choice(profile, voter) == winner])

print(f'\nPlurality rule winner: {winner}')
print(f'Top vote: {winner_on_top} times')
print(f'Bottom vote: {winner_on_bottom} times')



Preferences by frequency:
cba: 23   bac: 19   acb: 16   bca: 16   cab: 14   abc: 12   

Plurality rule winner: c
Top vote: 37 times
Bottom vote: 31 times


In the case of $|A|=3$ alternatives, the winner is despised by a 31 out of 100 voters, they put the winner $c$ at the bottom of their preferences. However, it shows that if the voters choosing $a$ or $b$ cooperated, and decided to only vote on *either* $a$ or $b$, they could outnumber $c$, as they would get 100 - 37 = 63 votes out of 100.

#### Plurality with 2 alternatives

In [173]:
random.seed(874)


n_voters = 100
n_alternatives = 2
voters = create_voters(n_voters)
alternatives = create_alternatives(n_alternatives)
profile = create_profile(voters, alternatives)

print('\nPreferences by frequency:')

preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

winner = plurality_rule(profile)
winner_on_top = sum([1 for voter in profile if top_choice(profile, voter) == winner])
winner_on_bottom = sum([1 for voter in profile if bottom_choice(profile, voter) == winner])

print(f'\nPlurality rule winner: {winner}')
print(f'Top vote: {winner_on_top} times')
print(f'Bottom vote: {winner_on_bottom} times')



Preferences by frequency:
ab: 54   ba: 46   

Plurality rule winner: a
Top vote: 54 times
Bottom vote: 46 times


In the case of just 2 alternatives, the plurality rule results in a winner that is on average despised by the largest fraction of the population. Compared to 3, 4 or 5 alternatives, the winner is at the bottom of the voter's preferences 46% of the time (!). 


## The plurarity with runoff rule $F_{RUN}$

The plurality with runoff rule seeks to fix the issue with the majority rule often not resulting in a winner if there are more than 2 alternatives. If there is no majority winner, pick the two alternatives with the highest plurality scores, and hold an extra round of voting between these two. Then, pick the majority winner out of these two alternatives.

$F_{RUN}(R) = F_{MAJ}(R)$ if it exists, otherwise $F_{RUN}(R) = F_{MAJ}(R')$

with $R'$ being the top two most popular candidates in $R$.

In [174]:
def runoff_rule(profile: dict) -> str | None:
    """
    This rule returns the majority rule winner.
    If no such winner exists, get the two most popular alternatives and hold
    another round of voting between these two, with the same preference profile,
    except with the candidates who lost in the first round omitted.
    """
    
    majority = len(profile) // 2 + 1

    top_choices = [top_choice(profile, voter) for voter in profile]
    top_choice_count = Counter(top_choices)

    for top, count in top_choice_count.items():
        if count >= majority:
            return top
    
    # if two tied for first place, no runoff winner
    if len(top_choice_count) <= 2:
        return None
    
    # if no majority winner, get the two most popular alternatives
    # for simplicity we do not introduce tie-breaking mechanisms here
    most_common = top_choice_count.most_common(2)
    top_2 = [a for a, _ in most_common]

    new_profile: dict[int, str] = dict()
    # per preference, remove all alternatives except the top 2
    for voter in profile:
        new_profile[voter] = ''.join([a for a in profile[voter] if a in top_2])
    
    # run majority rule again
    return majority_rule(new_profile)


In [175]:
random.seed(787)

n_voters = 10
n_alternatives = 3
profile = create_profile(create_voters(n_voters), create_alternatives(n_alternatives))
print(f'{n_voters=}')
print(f'{n_alternatives=}')

print('\nPreferences by frequency:')

preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

majority_winner = majority_rule(profile)
print(f'\nMajority rule winner: {majority_winner}')
runoff_winner = runoff_rule(profile)
print(f'Plurality with runoff winner: {runoff_winner}')

n_voters=10
n_alternatives=3

Preferences by frequency:
bca:  3   cab:  3   bac:  2   abc:  1   cba:  1   

Majority rule winner: None
Plurality with runoff winner: b


In the above simulation with 10 voters and 3 alternatives, there is no majority winner. By using the plurality with runoff rule, we can eliminate the least popular voters (in this case $a$), and retry using the adjusted preference profile with $a$ removed. This results in the winner $b$, as it is more often the case that $b \succ_i c$ than $c \succ_i b$ for the voters $i \in N$. When omitting $a$ from the above preference profile, it turns out that 3+2+1=6 voters prefer b over c, and 3+1=4 voters prefer c over b.

However, it is possible that a bad candidate is elected. For example: if 40 voters had the preference `acb`, 35 voters had `bca`, and 25 voters had `cba`, then after applying the plurality with runoff rule, $c$ would be eliminated, and $b$ would win. Note that in that scenario 65% of the voters initially preferred $c$ over $b$. 

## The Condorcet rule $F_{CON}$

Given that $n_{x,y}$ is the number of times that $x$ is preferred to $y$ in $R$, let $F_{CON}(R) = x$ such that $n_{x,y} > n_{y,x}$ for any other alternative $y \in A$. So, the Condercet winner is the alternative that beats any other alternative in a head-to-head contest. 

We implement the Condorcet rule by first constructing a $|A| \times |A|$ matrix $M$ in the function `condorcet_scores` containing the scores of each alternative matchup $n_{a_i, a_j}$ with $i,j \in [0, 1, ..., |A|]$. That is, $M_{i,j}$ is the amount of times alternative $a_i$ beats $a_j$ in a head-to-head contest. The matrix is filled with zeros along the diagonal since $a_i$ can never beat $a_i$.

In [176]:
def condorcet_scores(profile: dict[int, str]) -> np.ndarray:
    """
    Returns a matrix of Condorcet scores given a preference profile.
    """
    
    n_alternatives = len(profile[0])
    alternatives = list(string.ascii_lowercase[:n_alternatives])

    matchups = np.zeros((n_alternatives, n_alternatives), dtype=int)

    for preference in profile.values():
        for i, a_i in enumerate(alternatives):
            for j, a_j in enumerate(alternatives):
                # skip if same alternative
                if i == j:
                    continue

                # lower index means it beats the other
                if preference.index(a_i) < preference.index(a_j):
                    matchups[i, j] += 1
                
    return matchups


To illustrate this matrix, we construct we profile $R$ as shown in the lecture slides, and construct the matrix $M$:

In [177]:
n_voters = 13
n_alternatives = 4
print(f'{n_voters=}')
print(f'{n_alternatives=}')
submissions = ['abcd'] * 4 + ['bcda'] * 3 + ['cdba'] * 3 + ['dcba'] * 3

profile = {i: submissions[i] for i in range(n_voters)}

print('\nPreferences by frequency:')
preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

print('\nCondorcet scores matrix:')
print(condorcet_scores(profile))

n_voters=13
n_alternatives=4

Preferences by frequency:
abcd:  4   bcda:  3   cdba:  3   dcba:  3   

Condorcet scores matrix:
[[ 0  4  4  4]
 [ 9  0  7  7]
 [ 9  6  0 10]
 [ 9  6  3  0]]


Using this matrix, we can deduce that $b$ is the winner (also stated by the lecture slides). Counting from 1, row number $i = 2$ of $M$ indicates the score of $b$ when matched against opponents $j=1$ to $j=4$. $M_{2,2} = 0$ as stated previously, because $b$ cannot win against themselves. Based on the matrix $M$, $n_{b,a} > n_{a,b} \wedge n_{b,c} > n_{c,b} \wedge n_{b,d} > n_{d,b}$, thus $b$ is indeed the Condorcet winner. 

Below we construct the function `condorcet_rule` that calculates the Condorcet winner this way. By calling the function, we show that $b$ is indeed the winner for the given profile.

In [178]:
def condorcet_rule(profile) -> str | None:
    """
    This rule returns the Condorcet winner.
    If no such winner exists, it returns None.
    """
    
    matchups = condorcet_scores(profile)
    n_alternatives = len(matchups)

    # iterate over all rows in the matrix M
    for i in range(n_alternatives):

        # iterate over all matchups
        for j in range(n_alternatives):

            # skip if same alternative
            if i == j:
                continue

            # if there is a matchup where the alternative i loses or ties,
            # it is not a winner
            if matchups[i, j] <= matchups[j, i]:
                break

        else:
            # we did not break: the alternative never lost, so it wins
            return string.ascii_lowercase[i]
    
    # no Condorcet winner
    return None

In [179]:
winner = condorcet_rule(profile)
print(f'Condorcet rule winner: {winner}')


Condorcet rule winner: b


Keep in mind that there is not always a Condorcet winner:

In [180]:
random.seed(483)

n_voters = 20
n_alternatives = 4
profile = create_profile(create_voters(n_voters), create_alternatives(n_alternatives))
print(f'{n_voters=}')
print(f'{n_alternatives=}')

print('\nPreferences by frequency:')
preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

print('\nCondorcet scores matrix:')
matrix = condorcet_scores(profile)
print(matrix)

print('\nCondorcet rule winner:')
winner = condorcet_rule(profile)
print(f'{winner=}')

n_voters=20
n_alternatives=4

Preferences by frequency:
acdb:  3   cbad:  2   bacd:  2   cabd:  2   cbda:  2   cdab:  1   
dbac:  1   acbd:  1   adbc:  1   dbca:  1   dcba:  1   abcd:  1   
dabc:  1   cdba:  1   

Condorcet scores matrix:
[[ 0 10 10 12]
 [10  0  7 10]
 [10 13  0 15]
 [ 8 10  5  0]]

Condorcet rule winner:
winner=None


## The Borda rule $F_{BOR}$

The winner of the Borda rule is the alternative that accumulates the most points, which is done by being positioned in a favorable position in voter's preferences. For the Borda rule, the scoring scale is linear. That is, for $|A| = 3$ alternatives, the highest positioned alternative gets 2 points from a voter, the middle alternative gets 1 point and the least preferable alternative gets 0 points. In other words, alternatives gain points based on how many alternatives are positioned below them per voter preference.


$F_{BOR}(R) = \text{argmax}_{x \in A}b_R(x)$ with $b_i(x) = |A| - k$, if alternative $x$ is on the $k^{\text{th}}$ position in preference $\succ_i$ of agent $i$, and $b_R(x) = \Sigma_{i \in N} b_i(x)$. That is, the Borda rule chooses the alternative(s) with the highest Borda score(s).

The strength of this rule is that it uses more information than other rules. It takes into account the positioning of all alternatives listed in a preference. This makes this rule relatively more "fair". Additionally, Borda rule winner(s) always exist.

We first define the function `borda_scoring` that returns the array of scores $S$ with $S_i$ being the Borda score of alternative $a_i \in A$. 

In [181]:
def borda_scoring(profile: dict) -> np.ndarray:
    """
    Returns an array of Borda scores given a preference profile.
    """
    
    n_alternatives = len(profile[0])
    alternatives = list(string.ascii_lowercase[:n_alternatives])

    scores = np.zeros(n_alternatives, dtype=int)

    # iterate over all voter preferences
    for preference in profile.values():
        
        # iterate over all alternatives in the preference
        for i, a_i in enumerate(preference):
            
            # add the score for this alternative
            # - 1 because we start counting from 0
            scores[alternatives.index(a_i)] += n_alternatives - i - 1
    
    return scores

To illustrate this array, we construct we profile $R$ as shown in the lecture slides, and then construct the scores array $S$:

In [182]:
n_voters = 13
n_alternatives = 4
print(f'{n_voters=}')
print(f'{n_alternatives=}')
submissions = ['abcd'] * 4 + ['bcda'] * 3 + ['cdba'] * 3 + ['dcba'] * 3

profile = {i: submissions[i] for i in range(n_voters)}

print('\nPreferences by frequency:')
preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

print('\nBorda scores array:')
print(borda_scoring(profile))

n_voters=13
n_alternatives=4

Preferences by frequency:
abcd:  4   bcda:  3   cdba:  3   dcba:  3   

Borda scores array:
[12 23 25 18]


Based on this Borda scores array, we can deduce that the second alternative ($b$) is the Borda rule winner. This matches the result as shown on the lecture slides.

We now implement the function `scoring_rule` that actually computes the winner(s) and returns them using the provided scoring function (e.g. for the Borda rule we use the scoring function `borda_scoring`)

In [183]:
def scoring_rule(profile: dict, scoring_fn: callable) -> list[str]:
    """
    This rule returns the winner(s) specified by the scoring function.
    """

    scores = scoring_fn(profile)

    # if multiple winners, return all
    max_score = max(scores)
    winners = [a for a, s in zip(string.ascii_lowercase, scores) if s == max_score]

    return winners
    

To demonstrate this rule, we will again show the example from the lecture slides:

In [184]:
n_voters = 13
n_alternatives = 4
print(f'{n_voters=}')
print(f'{n_alternatives=}')
submissions = ['abcd'] * 4 + ['bcda'] * 3 + ['cdba'] * 3 + ['dcba'] * 3

profile = {i: submissions[i] for i in range(n_voters)}

print('\nPreferences by frequency:')
preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

winners = scoring_rule(profile, borda_scoring)
if winners:
    print(f'\nBorda rule winner(s): {", ".join(winners)}')
else:
    print('\nNo Borda rule winners.')

n_voters=13
n_alternatives=4

Preferences by frequency:
abcd:  4   bcda:  3   cdba:  3   dcba:  3   

Borda rule winner(s): c


However, the Borda rule is sensitive to the introduction or removal of new canditates. It is also not strategy-proof. That is, voters can lie about their true preferences in order to influence the winner to their advantage. We illustrate this below. By removing the alternative $c$, that has no chance of winning, we change the outcome of the election: suddenly $b$ is the winner instead of $a$.

In [185]:
n_voters = 100
n_alternatives = 3
submissions = ['acb'] * 35 + ['bac'] * 33 + ['cba'] * 32
profile = {i: submissions[i] for i in range(n_voters)}

print(f'{n_voters=}')
print(f'{n_alternatives=}')

print('\nPreferences by frequency:')
preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

scores = borda_scoring(profile)
print('\nBorda scores array:')
print(scores)

winners = scoring_rule(profile, borda_scoring)
if winners:
    print(f'\nBorda rule winner(s): {", ".join(winners)}')
else:
    print('\nNo Borda rule winners.')


n_voters=100
n_alternatives=3

Preferences by frequency:
acb: 35   bac: 33   cba: 32   

Borda scores array:
[103  98  99]

Borda rule winner(s): a


In [186]:
n_voters = 100
n_alternatives = 2
submissions = ['ab'] * 35 + ['ba'] * 33 + ['ba'] * 32
profile = {i: submissions[i] for i in range(n_voters)}

print(f'{n_voters=}')
print(f'{n_alternatives=}')

print('\nPreferences by frequency:')
preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

scores = borda_scoring(profile)
print('\nBorda scores array:')
print(scores)

winners = scoring_rule(profile, borda_scoring)
if winners:
    print(f'\nBorda rule winner(s): {", ".join(winners)}')
else:
    print('\nNo Borda rule winners.')


n_voters=100
n_alternatives=2

Preferences by frequency:
ba: 65   ab: 35   

Borda scores array:
[35 65]

Borda rule winner(s): b


## Various scoring rules

Finally, we will build on the Borda rule and propose various possible scoring rules. The Borda scores are assigned linearly, but what if you assign scores with more of a gap between the number 1 and the number 2? This is what the Downdall scoring method tries to capture.

In [187]:
def dowdall_scoring(profile: dict) -> np.ndarray:
    """
    Returns an array of Dowdall scores given a preference profile.
    """
    
    n_alternatives = len(profile[0])
    alternatives = list(string.ascii_lowercase[:n_alternatives])

    scores = np.zeros(n_alternatives)

    # iterate over all voter preferences
    for preference in profile.values():
        
        # iterate over all alternatives in the preference
        for i, a_i in enumerate(preference):
            
            # add the score for this alternative
            scores[alternatives.index(a_i)] += 1 / (i + 1)
    
    return scores

In [188]:
n_voters = 3
n_alternatives = 3
submissions = ['acb'] * 1 + ['bac'] * 1 + ['bca'] * 1
profile = {i: submissions[i] for i in range(n_voters)}

print(f'{n_voters=}')
print(f'{n_alternatives=}')

print('\nPreferences by frequency:')
preference_count = Counter(profile.values())

for i, (preference, count) in enumerate(preference_count.most_common()):
    # print in table form
    print(f'{preference}: {count:>2}', end='   ')
    if i % 6 == 5 and i != len(preference_count) - 1:
        print()

print()

scores = dowdall_scoring(profile)
print('\nDowdall scores array:')
print(scores)

winners = scoring_rule(profile, dowdall_scoring)
if winners:
    print(f'\nDowdall scoring winner(s): {", ".join(winners)}')
else:
    print('\nNo Dowdall scoring winners.')


n_voters=3
n_alternatives=3

Preferences by frequency:
acb:  1   bac:  1   bca:  1   

Dowdall scores array:
[1.83333333 2.33333333 1.33333333]

Dowdall scoring winner(s): b


Besides Dowdall, you can think of any scoring function possible. The scoring function $(1, 0, 0, ..., 0)$ is equivalent to just using the plurality rule $F_{PLU}$.

## Conclusion

We hope this overview clarifies the voting and scoring rules as discussed in the Social Choice lectures. Having the option to implement these voting and scoring rules in a programmatic way can help solve various issues in the field of Social Choice where voting or scoring rules are viable solutions.